In [2]:
import pandas as pd
from chembl_webresource_client.new_client import new_client

/Users/user/Downloads/DLF/GIT/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
#GRF = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF gtpase code\RF_molecules_Newfeatures_GTPase_KRas.csv")
#CRF = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF G12C Code\RF_molecules_Newfeatures_G12C.csv")
#DRF = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\RF G12D code\RF_molecules_Newfeatures_G12D.csv")
GNN = pd.read_csv("/Users/user/PycharmProjects/Drug Design FInal/FINAL_GIT/NN GTPase Code/NN_molecules_Newfeatures_GTPase_KRAS.csv")
#CNN = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\\NN G12C Code\\NN_molecules_Newfeatures_G12C.csv")
#DNN = pd.read_csv("C:\\Users\TheSh\Documents\Programming_in_Python_Class\PyCharmProjects\ML_for_KRAS_G12D_Inhibitors\\NN G12D Code\\NN_molecules_Newfeatures_G12D.csv")

In [4]:
df = GNN
df.head()

,Unnamed: 0,chembl_id,Predicted_Value
0,0,CHEMBL2146883,[np.float32(8.95349)]
1,1,CHEMBL3301603,[np.float32(7.759499)]
2,2,CHEMBL1200678,[np.float32(7.5652604)]
3,3,CHEMBL1668,[np.float32(7.5329623)]
4,4,CHEMBL3348963,[np.float32(7.1994147)]


In [8]:
list = []
for i in df["chembl_id"]:
    molecule = new_client.molecule
    m1 = molecule.filter(chembl_id=i).only(
        ['molecule_chembl_id', 'pref_name', 'molecule_structures'])
    targets = pd.DataFrame.from_dict(m1)
    #print(targets['molecule_chembl_id'][0])
    if targets['molecule_chembl_id'][0] == i:
        list.append([i,targets['pref_name'][0]])
        print(i,"&", targets['pref_name'][0],"&",df[df["chembl_id"]== i ]["Predicted_Value"],"\\\\\n \\hline")

CHEMBL2146883 & COBIMETINIB & 0    [np.float32(8.95349)]
Name: Predicted_Value, dtype: object \\
 \hline
CHEMBL3301603 & GILTERITINIB FUMARATE & 1    [np.float32(7.759499)]
Name: Predicted_Value, dtype: object \\
 \hline
CHEMBL1200678 & ATAZANAVIR SULFATE & 2    [np.float32(7.5652604)]
Name: Predicted_Value, dtype: object \\
 \hline
CHEMBL1668 & RESCINNAMINE & 3    [np.float32(7.5329623)]
Name: Predicted_Value, dtype: object \\
 \hline
CHEMBL3348963 & TELOTRISTAT ETIPRATE & 4    [np.float32(7.1994147)]
Name: Predicted_Value, dtype: object \\
 \hline
CHEMBL1201244 & ROCURONIUM & 5    [np.float32(6.9268174)]
Name: Predicted_Value, dtype: object \\
 \hline
CHEMBL6966 & VERAPAMIL & 6    [np.float32(6.3693757)]
Name: Predicted_Value, dtype: object \\
 \hline
CHEMBL282724 & SITAXENTAN & 7    [np.float32(5.7964606)]
Name: Predicted_Value, dtype: object \\
 \hline
CHEMBL2040682 & CICLESONIDE & 8    [np.float32(5.6234417)]
Name: Predicted_Value, dtype: object \\
 \hline
CHEMBL3218576 & COPANLIS

In [11]:
float(df[df["ChEMBL ID"]== "CHEMBL2361370" ]["Avg Predicted Value"])

C:\Users\TheSh\AppData\Local\Temp\ipykernel_12668\1317350521.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(df[df["ChEMBL ID"]== "CHEMBL2361370" ]["Avg Predicted Value"])


7.419709268156769